Pre Processing Netatmo Weather

In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from joblib import dump, load
from joblib import dump
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix

In [2]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
directory = r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\csv'
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [4]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

In [5]:
df_merged = merged_dataframe

In [6]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\own_data\DTADSV-S\mapping.xlsx', engine='openpyxl')

In [7]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.src_ip, mapping_df.target))

In [8]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['src_ip'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [9]:
flow_counts = df_merged.groupby('target')['Flow'].nunique()

In [10]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [11]:
df = df_merged

In [12]:
        df.replace('N/A', np.nan, inplace=True)
        df['Start_Time'] = pd.to_datetime(df['Start_Time'])
        df['End_Time'] = pd.to_datetime(df['End_Time'])
        df['Flow_Duration'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds()

        df['Flow_Bytes/s'] = df['Flow_Volume'] / df['Flow_Duration']
        df['Flow_Packets/s'] = df['Packet_Count'] / df['Flow_Duration']

        df.loc[df['Total_Fwd_Packets'] > 0, 'Fwd_IAT'] = df['Inter_Arrival_Time']
        df.loc[df['Total_Bwd_Packets'] > 0, 'Bwd_IAT'] = df['Inter_Arrival_Time']

        feature_df = df.groupby('Flow').agg({
            'Inter_Arrival_Time': ['mean', 'std', 'max', 'min'],
            'Total_Fwd_Packets': ['sum'],
            'Total_Bwd_Packets': ['sum'],
            'Fwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Bwd_Packet_Length_Total': ['max', 'min', 'mean', 'std'],
            'Flow_Bytes/s': ['mean'],
            'Flow_Packets/s': ['mean'],
            'packet_length': ['min', 'max', 'mean', 'std', 'var'],
            'Fwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
            'Bwd_IAT': ['sum', 'mean', 'std', 'max', 'min'],
        })

In [13]:
print("Columns in df: ", df.columns)
print("Columns in feature_df: ", feature_df.columns)

Columns in df:  Index(['idx', 'protocol', 'src_ip', 'src_port', 'dst_ip', 'dst_port',
       'packet_length', 'tcp_flags', 'window_size', 'ttl', 'frame_len',
       'dsfield_dscp', 'app_layer_protocol', 'dsfield_ecn', 'flags_df',
       'flags_mf', 'ip_hdr_len', 'ip_id', 'ip_len', 'udp_src_port',
       'udp_dst_port', 'udp_length', 'udp_checksum_status', 'packet_eth_src',
       'flow_label', 'traffic_class', 'next_header', 'payload_len',
       'packet.sniff_timestamp', 'Flow', 'Packet_Count', 'Start_Time',
       'End_Time', 'Duration', 'Flow_Volume', 'Flow_Rate',
       'Inter_Arrival_Time', 'Total_Fwd_Packets', 'Total_Bwd_Packets',
       'Fwd_Packet_Length_Total', 'Bwd_Packet_Length_Total', 'target',
       'Flow_Duration', 'Flow_Bytes/s', 'Flow_Packets/s', 'Fwd_IAT',
       'Bwd_IAT'],
      dtype='object')
Columns in feature_df:  MultiIndex([(     'Inter_Arrival_Time', 'mean'),
            (     'Inter_Arrival_Time',  'std'),
            (     'Inter_Arrival_Time',  'max'),
   

In [14]:
# Adjust column names and merge
feature_df.columns = ['_'.join(col) for col in feature_df.columns]
feature_df.reset_index(inplace=True)
df = pd.merge(df, feature_df, on='Flow', how='left')

-------------------------------------------------------------------------------------------

### **DTV-Set**

In [15]:
clf = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\rf_trained_model.joblib')

In [16]:
df_new = df

In [17]:
df_new.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,Fwd_IAT_sum,Fwd_IAT_mean,Fwd_IAT_std,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min
3683730,1562,TCP,154.52.17.92,443.0,192.168.1.99,8094.0,1308.0,ack,35840.0,51.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683731,1563,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,66.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683732,1564,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,375.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683733,1565,TCP,154.52.17.92,443.0,192.168.1.99,8094.0,66.0,ack,38912.0,51.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0
3683734,1566,TCP,192.168.1.99,8094.0,154.52.17.92,443.0,455.0,ack,26192.0,64.0,...,0.703296,0.04137,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0


In [18]:
# Count the number of packets for each target
target_counts = df_new['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            3603642
tp_link_c200         52627
netatmo_cam          26126
netatmo_weather       1340
Name: count, dtype: int64


In [19]:
# Drop duplicate rows based on the 'Flow' column, keeping only the first occurrence
df_new = df_new.drop_duplicates(subset=['Flow'], keep='first')

In [20]:
# Count the number of packets for each target
target_counts = df_new['target'].value_counts()

# Print the counts
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown            50566
tp_link_c200        4138
netatmo_cam          398
netatmo_weather       63
Name: count, dtype: int64


In [21]:
df_new = pd.get_dummies(df_new, columns=['protocol'])

In [22]:
df_new['ip_id'] = df_new['ip_id'].apply(lambda x: int(x, 16) if isinstance(x, str) else x)

In [23]:
df_new = df_new.fillna(0)

In [24]:
df_new.tail()

,idx,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,...,Fwd_IAT_std,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP
3683553,1385,192.168.1.112,60816.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,...,0.005893,0.014769,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683555,1387,192.168.1.112,60817.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,...,0.006364,0.014854,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683602,1434,192.168.1.112,60822.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,...,0.006495,0.018601,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683656,1488,192.168.1.112,60826.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,...,0.005478,0.012971,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683716,1548,192.168.1.99,8094.0,154.52.17.92,443.0,74.0,syn,14600.0,64.0,74.0,...,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0,True,False


In [25]:
from joblib import load
import pandas as pd

# Load reference headers
ref_headers = load(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\rf\rf_reference_header.joblib')

current_headers = df_new.columns

missing_headers = [col for col in ref_headers if col not in current_headers]

# Create a DataFrame containing the missing columns initialized to 0
df_missing = pd.DataFrame(0, index=df_new.index, columns=missing_headers)

# Concatenate the original and the new DataFrame along the columns
df_new = pd.concat([df_new, df_missing], axis=1)

# Reorder the DataFrame according to ref_headers
df_new = df_new[ref_headers]


In [26]:
df_new.tail()

,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,frame_len,dsfield_dscp,...,Fwd_IAT_std,Fwd_IAT_max,Fwd_IAT_min,Bwd_IAT_sum,Bwd_IAT_mean,Bwd_IAT_std,Bwd_IAT_max,Bwd_IAT_min,protocol_TCP,protocol_UDP
3683553,192.168.1.112,60816.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.005893,0.014769,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683555,192.168.1.112,60817.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.006364,0.014854,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683602,192.168.1.112,60822.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.006495,0.018601,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683656,192.168.1.112,60826.0,208.91.112.53,53.0,66.0,syn,64240.0,128.0,66.0,0.0,...,0.005478,0.012971,0.0,0.000000,0.00000,0.000000,0.000000,0.0,True,False
3683716,192.168.1.99,8094.0,154.52.17.92,443.0,74.0,syn,14600.0,64.0,74.0,0.0,...,0.068393,0.165497,0.0,0.703296,0.04137,0.068393,0.165497,0.0,True,False


In [27]:
X_new = df_new.drop(columns=[
    'target', 'src_ip', 'src_port', 'dst_ip', 'packet_eth_src', 'flow_label',
    'traffic_class', 'Flow', 'Start_Time', 'End_Time', 'next_header', 'ip_len',
    'udp_src_port', 'dsfield_ecn', 'dsfield_dscp', 'flags_mf', 'udp_checksum_status', 'udp_dst_port', 'udp_length','flags_df','ip_hdr_len',
    "packet.sniff_timestamp", "packet_length", "tcp_flags", "app_layer_protocol", "dst_port", "ttl", "Bwd_Packet_Length_Total", "Fwd_Packet_Length_Total",
    "Total_Bwd_Packets", "Total_Fwd_Packets", "ip_id", "Bwd_IAT_min", "packet_length_var", "payload_len", "Fwd_IAT_min", 
    "protocol_TCP", "protocol_UDP", "Flow_Duration", "Fwd_IAT_std", "Flow_Packets/s", "Bwd_Packet_Length_Total_mean"
    ])

In [28]:
Y_new = df_new.target

In [29]:
X_new = X_new.astype(float)

In [30]:
Y_new = Y_new.astype(str)

In [31]:
df = df.round(8)

In [32]:
X_new.fillna(0, inplace=True)

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
X_new[np.isinf(X_new)] = 0

In [35]:
Y_new_pred = clf.predict(X_new)

c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [36]:
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [37]:
new_accuracy = accuracy_score(Y_new, Y_new_pred)
print('Accuracy on new data:', new_accuracy)

Accuracy on new data: 0.5860781292486178


In [38]:
all_labels = ['amazon_echo', 'belkin_motion', 'belkin_switch', 'netatmo_cam', 'netatmo_weather', 'pixstar_photoframe', 'samsung_cam', 'smart_things', 'tp-link_cam', 'triby_speaker', 'unknown', 'tp_link_c200']
# Ensure a consistent order of labels
labels_ordered = list(set(all_labels))

In [39]:
cm = confusion_matrix(Y_new, Y_new_pred, labels=labels_ordered)

In [40]:
cm_df = pd.DataFrame(cm, index=labels_ordered, columns=labels_ordered)
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
                    tp_link_c200  tp-link_cam  netatmo_weather  belkin_motion  \
tp_link_c200                   0           34                0              2   
tp-link_cam                    0            0                0              0   
netatmo_weather                0            0               59              2   
belkin_motion                  0            0                0              0   
smart_things                   0            0                0              0   
netatmo_cam                    0            0                0             41   
samsung_cam                    0            0                0              0   
amazon_echo                    0            0                0              0   
pixstar_photoframe             0            0                0              0   
unknown                        0         1472                3           1241   
triby_speaker                  0            0                0              0   
belkin_swi

In [41]:
cm_df.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\rf\rf_DTADSV-S\rf_cm_DTADSV-Set.csv')

In [42]:
Y_new_df = pd.DataFrame(Y_new, columns=['Actual'])
Y_new_pred_df = pd.DataFrame(Y_new_pred, columns=['Predicted'])

In [43]:
result_df = pd.concat([Y_new_df, Y_new_pred_df], axis=1)

In [44]:
Y_test_pred = clf.predict(X_new)

c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [45]:
df_new['Predicted_Target'] = Y_new_pred

In [46]:
wrong_predictions = df_new[df_new['target'] != df_new['Predicted_Target']]

In [47]:
df_new['is_correct'] = df_new['target'] == df_new['Predicted_Target']

In [48]:
df_new.to_csv(r'C:\Users\Meyer\Documents\Dom_Detecter\def_ml\flow\rf\rf_DTADSV-S\rf_DTADSV-Set.csv', index=False)

In [49]:
# Group by the true label (i.e., the actual device type)
grouped = df_new.groupby('target')

In [50]:
accuracy_per_device = {}

In [51]:
# Calculate accuracy for each device type
for name, group in grouped:
    accuracy = accuracy_score(group['target'], group['Predicted_Target'])
    accuracy_per_device[name] = accuracy

In [52]:
print("Accuracy per device type:", accuracy_per_device)

Accuracy per device type: {'netatmo_cam': 0.6834170854271356, 'netatmo_weather': 0.9365079365079365, 'tp_link_c200': 0.0, 'unknown': 0.6328362931614128}
